<a href="https://colab.research.google.com/github/aqsamalic/LibraryManagement_assignment7/blob/main/LiabraryManagment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Classes and Inheritance:

In [9]:
# @title Default title text
from typing import List, Optional

# Base User class
class User:
    def __init__(self, user_id: int, name: str, email: str):
        self.user_id = user_id
        self.name = name
        self.email = email

    def display_info(self):
        print(f"User ID: {self.user_id}, Name: {self.name}, Email: {self.email}")

# Librarian class that inherits User
class Librarian(User):
    def __init__(self, user_id: int, name: str, email: str):
        super().__init__(user_id, name, email)

# Member class that inherits User
class Member(User):
    def __init__(self, user_id: int, name: str, email: str):
        super().__init__(user_id, name, email)
        self.borrowed_books: List[str] = []

    def borrow_book(self, book_id: int, library_manager):
        library_manager.borrow_book(self, book_id)

    def return_book(self, book_id: int, library_manager):
        library_manager.return_book(self, book_id)


2-Book Class:

In [10]:
class Book:
    def __init__(self, book_id: int, title: str, author: str, available: bool = True):
        self.book_id = book_id
        self.title = title
        self.author = author
        self.available = available

    def display_info(self):
        status = 'Available' if self.available else 'Borrowed'
        print(f"ID: {self.book_id}, Title: {self.title}, Author: {self.author}, Status: {status}")

    def borrow(self):
        if self.available:
            self.available = False
        else:
            raise Exception("Book is already borrowed!")

    def return_book(self):
        self.available = True


3. Library Management:

In [13]:
import os
import json

class LibraryManager:
    def __init__(self, book_file: str = 'books.txt', user_file: str = 'users.txt'):
        self.book_file = book_file
        self.user_file = user_file
        self.books: List[Book] = []
        self.users: List[User] = []
        self.load_books()
        self.load_users()

    # Load books from file
    def load_books(self):
        try:
            if os.path.exists(self.book_file):
                with open(self.book_file, 'r') as file:
                    book_data = json.load(file)
                    for item in book_data:
                        book = Book(**item)
                        self.books.append(book)
        except Exception as e:
            print(f"Error loading books: {e}")

    # Load users from file
    def load_users(self):
        try:
            if os.path.exists(self.user_file):
                with open(self.user_file, 'r') as file:
                    user_data = json.load(file)
                    for item in user_data:
                        user_type = item.pop('type')
                        if user_type == 'Librarian':
                            user = Librarian(**item)
                        else:
                            user = Member(**item)
                        self.users.append(user)
        except Exception as e:
            print(f"Error loading users: {e}")

    # Save books to file
    def save_books(self):
        try:
            with open(self.book_file, 'w') as file:
                book_data = [vars(book) for book in self.books]
                json.dump(book_data, file)
        except Exception as e:
            print(f"Error saving books: {e}")

    # Save users to file
    def save_users(self):
        try:
            with open(self.user_file, 'w') as file:
                user_data = []
                for user in self.users:
                    data = vars(user)
                    if isinstance(user, Librarian):
                        data['type'] = 'Librarian'
                    else:
                        data['type'] = 'Member'
                    user_data.append(data)
                json.dump(user_data, file)
        except Exception as e:
            print(f"Error saving users: {e}")

    # Add a new book (for librarians)
    def add_book(self, librarian: Librarian, book: Book):
        if isinstance(librarian, Librarian):
            self.books.append(book)
            self.save_books()
            print(f"Book '{book.title}' added successfully.")
        else:
            print("Only librarians can add books!")

    # Update book details (for librarians)
    def update_book(self, librarian: Librarian, book_id: int, title: Optional[str] = None, author: Optional[str] = None):
        if isinstance(librarian, Librarian):
            for book in self.books:
                if book.book_id == book_id:
                    if title:
                        book.title = title
                    if author:
                        book.author = author
                    self.save_books()
                    print(f"Book '{book.title}' updated successfully.")
                    return
            print(f"Book with ID {book_id} not found!")
        else:
            print("Only librarians can update books!")

    # Delete a book (for librarians)
    def delete_book(self, librarian: Librarian, book_id: int):
        if isinstance(librarian, Librarian):
            self.books = [book for book in self.books if book.book_id != book_id]
            self.save_books()
            print(f"Book with ID {book_id} deleted successfully.")
        else:
            print("Only librarians can delete books!")

    # Borrow a book (for members)
    def borrow_book(self, member: Member, book_id: int):
        for book in self.books:
            if book.book_id == book_id:
                if book.available:
                    book.borrow()
                    member.borrowed_books.append(book.title)
                    self.save_books()
                    print(f"{member.name} borrowed '{book.title}'")
                else:
                    print("Book is not available.")
                return
        print(f"Book with ID {book_id} not found.")

    # Return a book (for members)
    def return_book(self, member: Member, book_id: int):
        for book in self.books:
            if book.book_id == book_id and book.title in member.borrowed_books:
                book.return_book()
                member.borrowed_books.remove(book.title)
                self.save_books()
                print(f"{member.name} returned '{book.title}'")
                return
        print(f"Book with ID {book_id} not found or not borrowed by {member.name}.")
